In [322]:
import requests
import json
import sqlite3
import os
from dotenv import load_dotenv

## Project: looking at my top 100 songs
I am going to use Spotify's API to access 4 playlists: my top 100 songs from 2022, 2023, 2024, and 2025 (migrated from Apple Music). Then, I am going to create different SQL tables with to explore relationships between all of my favorite songs from the past 4 years.

In [ ]:
# # Request a bearer acess token given my Spotify client id / client secret

# Load environment variables from .env file
load_dotenv()

# Retrieve the credentials
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Request a bearer access token using the Spotify API
url = "https://accounts.spotify.com/api/token"
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
}

response = requests.post(url, data=data)
response_data = response.json()

# Check for errors
if "access_token" in response_data:
    authorization = f"Bearer {response_data['access_token']}"
    print("Access token received successfully!")
else:
    print("Error:", response_data)


# Using that authorization, access each playlist
# Turn it into text, then turn it into a dictorionary using json.loads

# For 2022
playlist_2022 = "https://api.spotify.com/v1/playlists/71zUjPqRk9OxTqpXYEmzlA/tracks"
playlist_2022_info = requests.get(url=playlist_2022, headers={"Authorization": authorization})
res_2022 = json.loads(playlist_2022_info.text)

# For 2023
playlist_2023 = "https://api.spotify.com/v1/playlists/28b3xd3RpAvarZB5PKr6Nx/tracks"
playlist_2023_info = requests.get(
    url=playlist_2023, headers={"Authorization": authorization}
)
res_2023 = json.loads(playlist_2023_info.text)

# For 2024
playlist_2024 = "https://api.spotify.com/v1/playlists/5P6ptyrOrEwoxn7A3qhC7F/tracks"
playlist_2024_info = requests.get(
    url=playlist_2024, headers={"Authorization": authorization}
)
res_2024 = json.loads(playlist_2024_info.text)

# For 2025 (so far)
playlist_2025 = "https://api.spotify.com/v1/playlists/0VLFZelqY8r0g544QAy8zv/tracks"
playlist_2025_info = requests.get(
    url=playlist_2025, headers={"Authorization": authorization}
)
res_2025 = json.loads(playlist_2025_info.text)

Access token received successfully!
{'href': 'https://api.spotify.com/v1/playlists/71zUjPqRk9OxTqpXYEmzlA/tracks?offset=0&limit=100', 'items': [{'added_at': '2025-03-16T03:28:07Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/602hgorenvoomy52d26p3c30u'}, 'href': 'https://api.spotify.com/v1/users/602hgorenvoomy52d26p3c30u', 'id': '602hgorenvoomy52d26p3c30u', 'type': 'user', 'uri': 'spotify:user:602hgorenvoomy52d26p3c30u'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': None, 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', '

### My SQL Plan:
- Create table PLAYLISTS to hold information about my four playlists (top 100 songs for 2022, 2023, 2024, 2025)
- Create table SONGS to hold information about the unique songs in all of these playlists
- Create table SONG ON PLAYLIST to relate the two tables above (song ID and playlist ID will relate this)

In [260]:
conn = sqlite3.connect('favorite_songs.db')
cursor = conn.cursor()

# cursor.execute("""CREATE TABLE playlists (
#                id text,
#                name text
#                )""")

# cursor.execute("INSERT INTO playlists VALUES ('71zUjPqRk9OxTqpXYEmzlA', 'Replay 2022')")
# cursor.execute("INSERT INTO playlists VALUES ('28b3xd3RpAvarZB5PKr6Nx', 'Replay 2023')")
# cursor.execute("INSERT INTO playlists VALUES ('5P6ptyrOrEwoxn7A3qhC7F', 'Replay 2024')")
# cursor.execute("INSERT INTO playlists VALUES ('0VLFZelqY8r0g544QAy8zv', 'Replay 2025')")

# conn.commit()

In [261]:
def get_song_info(playlist_res):
    """Given the dictionary created above, access all of this information
    about each song (should be 100 songs) on the playlist.
    Return the result as a list of tuples for each song"""

    # Holds all of the tuples, one tuple for each song
    song_info = []

    # Iterate through each song
    for i in range(0, len(playlist_res["items"])):
        song_name = playlist_res["items"][i]["track"]["name"]
        song_id = playlist_res["items"][i]["track"]["id"]
        song_artist = playlist_res["items"][i]["track"]["artists"][0]["name"]
        song_runtime = playlist_res["items"][i]["track"]["duration_ms"]
        song_popularity = playlist_res["items"][i]["track"]["popularity"]
        song_album = playlist_res["items"][i]["track"]["album"]["name"]

        # Tuple holding song's info gets appended
        full_info = (song_id, song_name, song_artist, song_runtime, song_popularity, song_album)
        song_info.append(full_info)

    return song_info

In [262]:
# cursor.execute(
#   """DROP TABLE Songs"""
# )

In [325]:
# cursor.execute(
#     """CREATE TABLE Songs (
#     id text,
#     name text,
#     artist text,
#     runtime real,
#     popularity integer,
#     album text
#     )"""
# )
# conn.commit()


def insert_song_info(playlist_res):
    for i in range(len(playlist_res["items"])):
        id, name, artist, runtime, popularity, album = get_song_info(playlist_res)[i]

        # Insert only if the song does not exist
        cursor.execute(
            """
            INSERT INTO Songs (id, name, artist, runtime, popularity, album)
            SELECT ?, ?, ?, ?, ?, ?
            WHERE NOT EXISTS (
                SELECT 1 FROM Songs WHERE id = ?
            )
            """,
            (id, name, artist, runtime, popularity, album, id),
        )

        conn.commit()
        # print(f"Inserted song {name}")


# Insert songs for multiple years
insert_song_info(res_2022)
insert_song_info(res_2023)
insert_song_info(res_2024)
insert_song_info(res_2025)

OperationalError: database is locked

In [ ]:
# cursor.execute(
#   """CREATE TABLE Songs_On_Playlist (
#   Song_ID text,
#   Playlist_ID text)"""
# )

def insert_id_info(playlist_res, playlist_id):
    """Insert song - playlist ids to establish relationship"""
    
    for i in range(len(playlist_res["items"])):
        song_id = get_song_info(playlist_res)[i][0]

        # Insert only if the song-playlist pair does not already exist
        cursor.execute(
            """
            INSERT INTO Songs_On_Playlist (song_id, playlist_id)
            SELECT ?, ?
            WHERE NOT EXISTS (
                SELECT 1 FROM Songs_On_Playlist WHERE song_id = ? AND playlist_id = ?
            )
            """,
            (song_id, playlist_id, song_id, playlist_id),
        )

        conn.commit()


# Insert song-playlist relationships
insert_id_info(res_2022, "71zUjPqRk9OxTqpXYEmzlA")
insert_id_info(res_2023, "28b3xd3RpAvarZB5PKr6Nx")
insert_id_info(res_2024, "5P6ptyrOrEwoxn7A3qhC7F")
insert_id_info(res_2025, "0VLFZelqY8r0g544QAy8zv")

### Note about genres
Unfortunately, Spotify's API doesn't give a genre for a specific song or album, but it does sometimes provide an array of genres given an artist.

In [ ]:
def get_artist_info(artist_id):
    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    artist_info = requests.get(
      url=artist_url, headers={"Authorization": authorization}
  )
    artist = json.loads(artist_info.text)["name"]
    genres = json.loads(artist_info.text)["genres"][0]   # Just grab top genre, if any
    popularity = json.loads(artist_info.text)["popularity"]
    return artist, genres, popularity

# Taylor Swift apprently has no genres
print(get_artist_info("06HL4z0CvFAxyc27GXpf02"))

# Drake has rap and hip hop
print(get_artist_info("3TVXtAsR1Inumwj472S9r4"))

('Taylor Swift', [], 97)
('Drake', ['rap', 'hip hop'], 97)


In [ ]:
# cursor.execute(
#   """CREATE TABLE artists (
#   artist text,
#   artist_id text,
#   top_genre text
#   popularity integer)"""
# )

def insert_artist_info(artist_id):
  artist, genre, popularity = get_artist_info(artist_id)
  cursor.execute(
            """
            INSERT INTO artists (artist, genre, popularity)
            SELECT ?, ?, ?
            WHERE NOT EXISTS (
                SELECT 1 FROM Songs_On_Playlist WHERE artist_id = ?
            )
            """,
            (artist, genre, popularity, artist_id),
        )
  conn.commit()


# Get the distinct artists
artists = cursor.execute(
  """SELECT Songs.artist
  FROM Songs
  """
)
distinct_artists = artists.fetchall()

# artists = cursor.execute(
#   """SELECT Songs.artist, artists.artist_id
#   FROM Songs, artists
#   JOIN Songs So ON So.artist = artists.artist"""
# )

for artist in distinct_artists:
  insert_artist_info


[('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Norah Jones',),
 ('Taylor Swift',),
 ('Norah Jones',),
 ('Taylor Swift',),
 ('Rihanna',),
 ('Norah Jones',),
 ('Norah Jones',),
 ('Taylor Swift',),
 ('Drake',),
 ('Drake',),
 ('Taylor Swift',),
 ('The Lumineers',),
 ('Tyler Childers',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Kanye West',),
 ('Harry Styles',),
 ('Drake',),
 ('Drake',),
 ('Harry Styles',),
 ('Taylor Swift',),
 ('Drake',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Harry Styles',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Taylor Swift',),
 ('Counting Crows',),
 ('Taylor Swift',),
 ('Ariana Grande',),
 ('Taylor Swift',),
 ('Norah Jones',),
 ('Taylor Swift',),
 ('Norah Jones',),
 ('Mazzy Star',),
 ('The Lumineers',),
 ('Taylor Swift',),
 ('

## Exploration
Now that I have these three tables, I would like to do some exploration.

In [ ]:
# Get the average popularity of
popularity = cursor.execute(
  """SELECT Playlists.name AS playlist_name, AVG(Songs.popularity) AS avg_popularity
  FROM Songs
  JOIN Songs_On_Playlist ON Songs.id = Songs_On_Playlist.Song_ID
  JOIN Playlists ON Songs_On_Playlist.Playlist_ID = Playlists.id
  GROUP BY Playlists.id
  ORDER BY avg_popularity DESC"""
)
popularity.fetchall()

[('Replay 2022', 60.11),
 ('Replay 2023', 54.16),
 ('Replay 2024', 49.505050505050505),
 ('Replay 2025', 45.38)]

This result suggests that when I was first getting into music in 2022, I listened to more mainstream songs that are still popular today. Then as the years went on, I started refining my music taste.

In [ ]:
# What songs have been on my replay since 2022?
# Basically, I want the songs which have a repeat entry in Songs_On_Playlist (same SongID, different playlistID)
consistent_songs = cursor.execute(
    """SELECT Songs.name, Songs.artist, Songs.popularity, COUNT(*) as Freq
  FROM Songs
  JOIN Songs_On_Playlist ON Songs.id = Songs_On_Playlist.Song_ID
  GROUP BY Songs_On_Playlist.Song_ID
  ORDER BY Freq DESC"""
)
consistent_songs.fetchmany(15)

[('First Time', 'Hozier', 54, 3),
 ('Autumn Leaves - Album Version - (Take 1)', 'Bill Evans Trio', 35, 3),
 ('Jackie And Wilson', 'Hozier', 64, 3),
 ('august', 'Taylor Swift', 35, 3),
 ('cowboy like me', 'Taylor Swift', 62, 3),
 ('Part Of The Band', 'The 1975', 52, 3),
 ('Wintering', 'The 1975', 52, 3),
 ('Happiness', 'The 1975', 61, 3),
 ('Drops of Jupiter (Tell Me)', 'Train', 80, 3),
 ("September In The Rain - Live At Mister Kelly's, Chicago / 1957",
  'Sarah Vaughan',
  22,
  3),
 ('Daylight', 'Taylor Swift', 78, 3),
 ('About You', 'The 1975', 84, 3),
 ('Lover', 'Taylor Swift', 84, 3),
 ('All I Need To Hear', 'The 1975', 58, 3),
 ('Cruel Summer', 'Taylor Swift', 88, 3)]

In [272]:
# What is the average length of the songs I like?
# Ms to S = divide Ms by 60,000
song_lengths = cursor.execute(
  """SELECT AVG(Songs.runtime)/60000
  FROM Songs"""
)

song_lengths.fetchone()

(4.197860273224044,)

In [ ]:
# How many distinct albums did I listen to?
# How many times did I have a favorite from a certain album?
# Note to self: HAVING is for use after aggregation (WHERE is not for aggregation)
num_albums = cursor.execute(
    """SELECT Songs.album, COUNT(*) as album_freq
       FROM Songs
       GROUP BY Songs.album
       HAVING album_freq > 2
       ORDER BY album_freq DESC
    """
)
num_albums.fetchall()

[('Midnights (3am Edition)', 12),
 ('Lover', 10),
 ('Being Funny In A Foreign Language', 10),
 ('folklore (deluxe version)', 9),
 ('reputation', 8),
 ('eternal sunshine (slightly deluxe)', 7),
 ('Unreal Unearth', 7),
 ('The 1975 (Deluxe)', 7),
 ("Red (Taylor's Version)", 6),
 ('I like it when you sleep, for you are so beautiful yet so unaware of it',
  6),
 ("Harry's House", 6),
 ('Loveseat', 5),
 ('the record', 4),
 ('A Brief Inquiry Into Online Relationships', 4),
 ('1989 (Deluxe)', 4),
 ('1989', 4),
 ('SABLE,', 3),
 ("Fearless (Taylor's Version)", 3),
 ('Come Away With Me (Remastered 2022)', 3),
 ('Certified Lover Boy', 3)]

In [301]:
# I want to look at how my listening of Taylor Swift has changed over the years
taylor_swift = cursor.execute(
    """SELECT playlists.name, Songs.artist, COUNT(*)
       FROM Songs
       JOIN Songs_On_Playlist sop ON Songs.id = sop.Song_ID
       JOIN playlists ON playlists.id = sop.Playlist_ID
       WHERE Songs.artist = 'Taylor Swift'
       GROUP BY playlists.name
    """
)

taylor_swift.fetchall()

[('Replay 2022', 'Taylor Swift', 60),
 ('Replay 2023', 'Taylor Swift', 24),
 ('Replay 2024', 'Taylor Swift', 6),
 ('Replay 2025', 'Taylor Swift', 2)]